#  대화 이력 관리를 위한 메모리 구현(Chat History)

### **학습 목표:**  대화 이력 관리를 위한 메모리 컴포넌트 구현 방법을 실습한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9, 
)

# **채팅 히스토리 관리**

* 채팅봇의 핵심 기능은 이전 대화 내용을 문맥으로 활용하는 것으로, 이를 통해 자연스러운 대화 흐름을 유지할 수 있습니다.

* 가장 기본적인 방식은 이전 메시지들을 모델의 프롬프트에 직접 포함시키는 것이지만, 오래된 메시지는 적절히 제거하여 모델이 처리해야 할 정보량을 조절할 수 있습니다.

* 장시간 진행되는 대화의 경우, 단순히 이전 메시지를 저장하는 것을 넘어 대화 내용을 요약하여 저장하는 등의 고급 메모리 관리 기법을 활용할 수 있습니다.

### 1. **메시지 전달 방식(Message Passing)**

* 메시지 전달 방식은 LangChain에서 가장 기본적인 메모리 구현 방법으로, 이전 대화 기록(chat history)을 체인에 직접 전달하여 문맥을 유지하는 방식입니다.

* 이 방식은 SystemMessage(시스템 지시사항), HumanMessage(사용자 입력), AIMessage(AI 응답) 등 다양한 유형의 메시지를 ChatPromptTemplate을 통해 구조화하며, MessagesPlaceholder를 사용하여 이전 대화 내용을 포함시킵니다.

* 챗봇의 기본적인 메모리 시스템을 구현하는데 사용되며, 이를 통해 AI는 이전 대화 맥락을 이해하고 그에 맞는 적절한 응답을 생성할 수 있습니다.


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ChatPromptTemplate를 사용하여 챗봇의 초기 메시지를 정의
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="messages"),    # 메시지 목록을 동적으로 삽입하는 부분
])

# ChatPromptTemplate에 삽입할 메시지 목록을 정의
messages = [
        HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
        AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
]    

# ChatPromptTemplate에 삽입할 메시지 목록을 업데이트하고 출력 
pprint(prompt.format(messages=messages))

('System: You are a helpful assistant.\n'
 'Human: 안녕하세요. 제 이름은 홍길동입니다.\n'
 'AI: 안녕하세요! 어떻게 도와드릴까요?')


In [5]:
# 대화형 체인을 정의 (prompt -> llm)
chain = prompt | llm

# 기본적인 메시지 전달: 이전 메시지 목록에 새로운 메시지를 추가해서 전달
response = chain.invoke({
    "messages": messages + [HumanMessage(content="제 이름을 기억하나요?")] # 이전 메시지를 기억하는지 확인하는 질문 메시지 추가
})

pprint(response)

AIMessage(content='네, 제 이름은 홍길동이라고 하셨습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 53, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CEauR7fcxMl5gPkpqC1fwmu0yGgjp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--dd495f2d-c69b-45b3-be66-54bea7870bb5-0', usage_metadata={'input_tokens': 53, 'output_tokens': 20, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


### 2. **RunnableWithMessageHistory**

* RunnableWithMessageHistory는 LangChain에서 대화 기록을 관리하는 고급 기능으로, 체인의 실행 과정에서 메시지 기록을 자동으로 저장하고 검색할 수 있게 해주는 래퍼(wrapper) 클래스입니다.

* 이 기능은 대화 세션별로 독립적인 기록을 유지할 수 있게 해주며, ConfigurableField를 통해 메모리 구성을 유연하게 조정할 수 있습니다. 특히 여러 사용자와 동시에 대화할 때 각 세션의 컨텍스트를 분리하여 관리하는 데 매우 유용합니다.

* 주요 장점은 대화 기록 관리의 자동화와 일관성 있는 메시지 처리이지만, 메모리 저장소 설정과 관리에 추가적인 구성이 필요하다는 점을 고려해야 합니다. 또한 Redis나 다른 외부 저장소와 통합하여 영구적인 대화 기록 보관도 가능합니다.

* 구현 시에는 get_session_history 콜백을 통해 세션ID별로 메시지 기록을 관리하며, 이를 통해 각 대화의 컨텍스트를 정확하게 유지할 수 있습니다.


`(1) 메모리 기반 로컬 저장소 활용`

* `InMemoryHistory` 클래스는 대화 이력의 기본 구조를 제공하며, BaseChatMessageHistory와 BaseModel을 상속받아 메시지를 메모리에서 효율적으로 관리합니다. 특히 messages 리스트를 통해 BaseMessage 객체들을 순차적으로 저장하고, add_messages와 clear 메서드로 히스토리를 유연하게 관리할 수 있습니다.

* 시스템의 핵심인 `store` 변수는 전역 딕셔너리로 구현되어 세션별 대화 이력을 구분하여 저장합니다. session_id를 키로 사용하여 각 세션의 InMemoryHistory 객체에 빠르게 접근할 수 있으며, 이를 통해 다중 사용자 환경에서도 효율적인 대화 관리가 가능합니다.

* `get_session_history` 함수는 세션 관리의 진입점 역할을 하며, 존재하지 않는 세션에 대해 자동으로 새로운 InMemoryHistory 객체를 생성하는 팩토리 패턴을 구현합니다. 이러한 구조를 통해 세션의 생명주기를 자동으로 관리하고 메모리 효율성을 높일 수 있습니다.

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요.
2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 한국의 옛 생활 문화를 체험할 수 있습니다.
3. 명동 – 쇼핑과 길거리 음식, 다양한 카페가 모여 있는 활기찬 번화가입니다.
4. N서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 야경이 특히 아름답습니다.
5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 다양한 먹거리와 카페가 많아요.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 패션, 전시가 어우러진 문화 공간입니다.
7. 한강공원 – 강변을 따라 산책하거나 자전거를 탈 수 있고, 피크닉 장소로도 좋아요.

필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴 수 있습니다!


In [8]:
# 대화 히스토리 출력 
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요.\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 한국의 옛 생활 문화를 체험할 수 있습니다.\n3. 명동 – 쇼핑과 길거리 음식, 다양한 카페가 모여 있는 활기찬 번화가입니다.\n4. N서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 야경이 특히 아름답습니다.\n5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 다양한 먹거리와 카페가 많아요.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 패션, 전시가 어우러진 문화 공간입니다.\n7. 한강공원 – 강변을 따라 산책하거나 자전거를 탈 수 있고, 피크닉 장소로도 좋아요.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 252, 'prompt_tokens': 40, 'total_tokens': 292, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEauZNJeM15fBPTV

In [9]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.

- **경복궁**은 한국의 대표적인 역사 문화 유적지로, 많은 관광객이 한국 전통문화를 체험하기 위해 방문합니다. 특히, 수문장 교대식과 아름다운 궁궐 건축이 큰 매력입니다.
- **N서울타워**는 서울의 랜드마크 중 하나로, 서울 전경과 야경을 감상하려는 관광객들에게 매우 인기 있습니다. 특히 저녁 시간대에 방문하면 멋진 야경과 함께 로맨틱한 분위기를 즐길 수 있습니다.

두 곳 모두 서울 관광에서 꼭 가봐야 할 명소로 꼽히며, 방문객 수가 많아 인기 있는 장소입니다. 여행 일정에 맞춰 선택하시면 좋을 것 같아요!


In [10]:
# 대화 히스토리 출력
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요.\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 한국의 옛 생활 문화를 체험할 수 있습니다.\n3. 명동 – 쇼핑과 길거리 음식, 다양한 카페가 모여 있는 활기찬 번화가입니다.\n4. N서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 야경이 특히 아름답습니다.\n5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 다양한 먹거리와 카페가 많아요.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 패션, 전시가 어우러진 문화 공간입니다.\n7. 한강공원 – 강변을 따라 산책하거나 자전거를 탈 수 있고, 피크닉 장소로도 좋아요.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 252, 'prompt_tokens': 40, 'total_tokens': 292, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEauZNJeM15fBPTV

`(2) SQLite 데이터베이스 영구 저장소 활용`

* SQLite 통합 구현을 위해서는 먼저 메시지를 저장할 데이터베이스 테이블 구조를 정의하고, BaseChatMessageHistory를 상속받아 메시지 저장/조회 로직을 구현해야 합니다. 이때 세션 ID를 기준으로 대화 내용을 구분하여 관리합니다.

* 시스템 구현 시에는 메시지의 타입(Human/AI), 내용, 메타데이터, 타임스탬프 등의 정보를 체계적으로 저장하고, 필요할 때 효율적으로 검색하고 활용할 수 있도록 적절한 인덱싱 전략을 수립하는 것이 중요합니다.

In [11]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import sqlite3
from typing import List
import json

class SQLiteChatMessageHistory(BaseChatMessageHistory):
    """ 
    SQLite 데이터베이스를 사용하여 챗봇 대화 히스토리를 저장하는 클래스

    Attributes:
        session_id (str): 세션 ID
        db_path (str): SQLite 데이터베이스 파일 경로

    """
    def __init__(self, session_id: str, db_path: str = "chat_history.db"):
        self.session_id = session_id
        self.db_path = db_path
        self._create_tables()
    
    def _create_tables(self):
        """데이터베이스 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 메시지 테이블 생성
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT,
                message_type TEXT,
                content TEXT,
                metadata TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """단일 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO messages (session_id, message_type, content, metadata)
            VALUES (?, ?, ?, ?)
        """, (
            self.session_id,
            message.__class__.__name__,
            message.content,
            json.dumps(message.additional_kwargs)
        ))
        
        conn.commit()
        conn.close()
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """여러 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for message in messages:
            cursor.execute("""
                INSERT INTO messages (session_id, message_type, content, metadata)
                VALUES (?, ?, ?, ?)
            """, (
                self.session_id,
                message.__class__.__name__,
                message.content,
                json.dumps(message.additional_kwargs)
            ))
        
        conn.commit()
        conn.close()
    
    def clear(self) -> None:
        """세션의 모든 메시지 삭제"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM messages WHERE session_id = ?
        """, (self.session_id,))
        
        conn.commit()
        conn.close()
    
    @property
    def messages(self) -> List[BaseMessage]:
        """저장된 메시지 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, metadata
            FROM messages 
            WHERE session_id = ?
            ORDER BY created_at
        """, (self.session_id,))
        
        messages = []
        for row in cursor.fetchall():
            message_type, content, metadata = row
            if message_type == "HumanMessage":
                message = HumanMessage(content=content)
            else:
                message = AIMessage(content=content)
            
            if metadata:
                message.additional_kwargs = json.loads(metadata)
            
            messages.append(message)
        
        conn.close()
        return messages
    

# 세션 ID로 히스토리 가져오기
def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    return SQLiteChatMessageHistory(session_id=session_id)

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "수원에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
수원은 역사와 현대가 어우러진 매력적인 도시입니다. 수원에서 가볼 만한 명소를 몇 곳 추천해드릴게요.

1. 수원 화성  
- 조선 정조 시대에 세워진 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁, 장안문, 팔달문 등을 둘러보세요. 야경도 매우 아름답습니다.

2. 화성행궁  
- 조선시대 임시 왕궁으로 사용된 곳입니다. 전통 건축물과 함께 다양한 문화행사와 체험 프로그램이 열립니다.

3. 팔달문 시장  
- 수원의 대표 전통시장으로, 맛있는 길거리 음식과 다양한 쇼핑을 즐길 수 있습니다.

4. 경기상상캠퍼스  
- 옛 공장을 리모델링한 복합 문화 공간으로, 전시, 공연, 카페 등이 있어 젊은 층에게 인기입니다.

5. 광교 호수공원  
- 광교 신도시에 위치한 큰 호수공원으로, 산책과 자전거 타기, 피크닉을 즐기기에 좋습니다.

수원은 대중교통이 잘 발달되어 있어 이동도 편리합니다. 즐거운 여행 되세요!


In [13]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 현대가 어우러진 매력적인 도시입니다. 수원에서 가볼 만한 명소를 몇 곳 추천해드릴게요.\n\n1. 수원 화성  \n- 조선 정조 시대에 세워진 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁, 장안문, 팔달문 등을 둘러보세요. 야경도 매우 아름답습니다.\n\n2. 화성행궁  \n- 조선시대 임시 왕궁으로 사용된 곳입니다. 전통 건축물과 함께 다양한 문화행사와 체험 프로그램이 열립니다.\n\n3. 팔달문 시장  \n- 수원의 대표 전통시장으로, 맛있는 길거리 음식과 다양한 쇼핑을 즐길 수 있습니다.\n\n4. 경기상상캠퍼스  \n- 옛 공장을 리모델링한 복합 문화 공간으로, 전시, 공연, 카페 등이 있어 젊은 층에게 인기입니다.\n\n5. 광교 호수공원  \n- 광교 신도시에 위치한 큰 호수공원으로, 산책과 자전거 타기, 피크닉을 즐기기에 좋습니다.\n\n수원은 대중교통이 잘 발달되어 있어 이동도 편리합니다. 즐거운 여행 되세요!', additional_kwargs={'refusal': None}, response_metadata={})]


In [14]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천드린 장소 중 가장 인기 있는 곳은 **수원 화성**입니다.  

수원 화성은 역사적 가치가 뛰어나고, 아름다운 성곽과 함께 다양한 볼거리와 체험거리가 많아 관광객들에게 매우 사랑받는 명소입니다. 특히 성곽을 따라 걷는 산책로와 야경이 유명하며, 매년 많은 국내외 관광객이 방문합니다.  

화성행궁도 인기가 높지만, 수원 화성 전체를 둘러볼 수 있는 성곽과 주변 시설들이 더 많은 방문객을 끌어모으고 있습니다.  

수원 여행 시 꼭 방문해보시길 추천드립니다!


In [15]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 현대가 어우러진 매력적인 도시입니다. 수원에서 가볼 만한 명소를 몇 곳 추천해드릴게요.\n\n1. 수원 화성  \n- 조선 정조 시대에 세워진 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁, 장안문, 팔달문 등을 둘러보세요. 야경도 매우 아름답습니다.\n\n2. 화성행궁  \n- 조선시대 임시 왕궁으로 사용된 곳입니다. 전통 건축물과 함께 다양한 문화행사와 체험 프로그램이 열립니다.\n\n3. 팔달문 시장  \n- 수원의 대표 전통시장으로, 맛있는 길거리 음식과 다양한 쇼핑을 즐길 수 있습니다.\n\n4. 경기상상캠퍼스  \n- 옛 공장을 리모델링한 복합 문화 공간으로, 전시, 공연, 카페 등이 있어 젊은 층에게 인기입니다.\n\n5. 광교 호수공원  \n- 광교 신도시에 위치한 큰 호수공원으로, 산책과 자전거 타기, 피크닉을 즐기기에 좋습니다.\n\n수원은 대중교통이 잘 발달되어 있어 이동도 편리합니다. 즐거운 여행 되세요!', additional_kwargs={'refusal': None}, response_metadata={}),
 HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이전에 추천드린 장소 중 가장 인기 있는 곳은 **수원 화성**입니다.  \n\n수원 화성은 역사적 가치가 뛰어나고, 아름다운 성곽과 함께 다양한 볼거리와 체험거리가 많아 관광객들에게 매우 사랑받는 명소입니다. 특히 성곽을 따라 걷는 산책로와 야경이 유명하며, 매년 많은 국내외 관광객이 방문합니다.  \n\n화성행궁도 인기가 높지만, 수원 화성 전체를 둘러볼 수 있는

In [16]:
# 대화 히스토리 초기화
history.clear()

In [17]:
# 대화 히스토리 출력
pprint(history.messages)

[]


# **메시지 관리 기법**

* 메시지 트리밍은 컨텍스트 윈도우의 토큰 제한을 관리하며, 시스템 메시지 포함 여부와 시작 위치 등을 세밀하게 제어할 수 있습니다.

* 장시간 진행되는 대화의 경우, 이전 대화 내용을 한 문장으로 요약하여 컨텍스트로 활용함으로써 메모리 효율성을 높일 수 있습니다.

* 대화 히스토리가 일정 길이를 초과할 경우, 요약된 내용과 최근 메시지만을 새로운 히스토리로 구성하여 컨텍스트의 품질을 유지하면서도 토큰 사용량을 최적화할 수 있습니다.

### 1. **메시지 트리밍(Message Trimming)**

* `trim_messages` 함수는 컨텍스트 윈도우의 토큰 제한을 관리하기 위한 핵심 도구로, 시스템 메시지를 포함할지 여부와 어디서부터 트리밍을 시작할지 등을 상세하게 설정할 수 있습니다.

* 트리밍 전략으로 "last" 옵션을 사용하면 가장 최근의 메시지부터 시작하여 지정된 토큰 제한에 맞춰 이전 메시지들을 선택적으로 포함시킬 수 있습니다.

In [18]:
from langchain_core.messages import trim_messages

# 메시지 목록을 정의
orginal_messages = [
    HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
    AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
    HumanMessage(content="제 이름을 기억하나요?")
]

# 트리머(trimmer) 생성
# 마지막 메시지 2개만 유지하도록 설정 (token은 메시지를 나타냄) 
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

trimmed_messages = trimmer.invoke(orginal_messages)

pprint(trimmed_messages)

[AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 기억하나요?', additional_kwargs={}, response_metadata={})]


In [19]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field
from typing import List

# 메시지 트리밍이 적용된 인메모리 히스토리 구현
class TrimmedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    max_tokens: int = Field(default=2)  # 유지할 최대 메시지 수
    
    def __init__(self, max_tokens: int = 2, **kwargs):
        """
        TrimmedInMemoryHistory 초기화
        
        Args:
            max_tokens (int): 유지할 최대 메시지 수
            **kwargs: 추가 키워드 인자
        """
        super().__init__(max_tokens=max_tokens, **kwargs)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
        # 메시지 추가 후 트리밍 수행
        trimmer = trim_messages(
            strategy="last",
            max_tokens=self.max_tokens,
            token_counter=len
        )
        self.messages = trimmer.invoke(self.messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 트리밍된 히스토리 가져오기
def get_trimmed_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = TrimmedInMemoryHistory()
    return store[session_id]

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 트리밍된 히스토리 관리 추가
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain,
    get_trimmed_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있습니다. 한복 체험도 인기예요.
2. 북촌 한옥마을 – 전통 한옥이 모여 있는 마을로, 골목길을 걸으며 옛 서울의 분위기를 느낄 수 있습니다.
3. 명동 – 쇼핑과 길거리 음식을 즐길 수 있는 번화가로, 다양한 브랜드 매장과 맛집이 많아요.
4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있고 야경이 특히 아름답습니다.
5. 홍대 – 젊음의 거리로, 예술과 음악, 카페 문화가 활발한 곳입니다. 거리 공연과 독특한 상점들이 많아요.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 패션, 디자인 전시가 열리는 복합 문화 공간입니다.
7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 다양한 야외 활동을 즐길 수 있습니다.

필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴게요!


In [20]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있습니다. 한복 체험도 인기예요.\n2. 북촌 한옥마을 – 전통 한옥이 모여 있는 마을로, 골목길을 걸으며 옛 서울의 분위기를 느낄 수 있습니다.\n3. 명동 – 쇼핑과 길거리 음식을 즐길 수 있는 번화가로, 다양한 브랜드 매장과 맛집이 많아요.\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있고 야경이 특히 아름답습니다.\n5. 홍대 – 젊음의 거리로, 예술과 음악, 카페 문화가 활발한 곳입니다. 거리 공연과 독특한 상점들이 많아요.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 패션, 디자인 전시가 열리는 복합 문화 공간입니다.\n7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 다양한 야외 활동을 즐길 수 있습니다.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 40, 'total_tokens': 339, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-1

In [21]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.

- **경복궁**은 한국의 역사와 문화를 체험할 수 있는 대표적인 관광지로, 특히 외국인 관광객과 한복 체험을 원하는 분들에게 매우 인기가 많습니다.
- **N서울타워**는 서울의 전경을 한눈에 볼 수 있는 명소로, 낮과 밤 모두 방문객이 많으며, 특히 야경이 아름다워 데이트 코스로도 유명합니다.

두 곳 모두 서울을 처음 방문하는 관광객에게 꼭 추천되는 명소입니다. 방문 목적에 따라 선택하시면 좋을 것 같아요!


In [22]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.\n\n- **경복궁**은 한국의 역사와 문화를 체험할 수 있는 대표적인 관광지로, 특히 외국인 관광객과 한복 체험을 원하는 분들에게 매우 인기가 많습니다.\n- **N서울타워**는 서울의 전경을 한눈에 볼 수 있는 명소로, 낮과 밤 모두 방문객이 많으며, 특히 야경이 아름다워 데이트 코스로도 유명합니다.\n\n두 곳 모두 서울을 처음 방문하는 관광객에게 꼭 추천되는 명소입니다. 방문 목적에 따라 선택하시면 좋을 것 같아요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 363, 'total_tokens': 515, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEavdKi9rhG75XWBAGE7ObLmVEWtV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c14ea815-7fd0-4df1-90c1-8fd0

In [23]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이 장소의 정식 명칭은 무엇인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
경복궁의 정식 명칭은 **경복궁(景福宮, Gyeongbokgung Palace)**이며, 조선 시대의 법궁(왕이 거주하던 궁궐)입니다.

N서울타워의 정식 명칭은 **N서울타워(N Seoul Tower)** 또는 **남산서울타워(Namsan Seoul Tower)**로, 서울 남산에 위치한 통신 및 전망 타워입니다.

필요하신 추가 정보가 있으면 언제든 말씀해 주세요!


In [24]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이 장소의 정식 명칭은 무엇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='경복궁의 정식 명칭은 **경복궁(景福宮, Gyeongbokgung Palace)**이며, 조선 시대의 법궁(왕이 거주하던 궁궐)입니다.\n\nN서울타워의 정식 명칭은 **N서울타워(N Seoul Tower)** 또는 **남산서울타워(Namsan Seoul Tower)**로, 서울 남산에 위치한 통신 및 전망 타워입니다.\n\n필요하신 추가 정보가 있으면 언제든 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 217, 'total_tokens': 327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CEavg1RdXefCRG19wzdDqxMtxoFuS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8027e29d-51fb-451b-9ff7-a0ac8f6c6e2a-0', usage_metadata={'input_tokens': 217, 'output_tokens': 110, 'total_tokens': 327, 

### 2. **대화 요약 저장**

* 대화가 길어질 경우, 전체 대화 내용을 요약하여 컨텍스트로 활용하는 방식으로 이전 대화의 핵심을 추출합니다.

* 일반적으로 메시지 히스토리가 지정된 길이(예: 4개의 메시지)를 초과할 경우, 이전 대화들을 요약하고 가장 최근의 메시지만 유지하는 방식으로 새로운 대화 히스토리를 구성합니다.

* 이러한 요약 메모리 방식을 통해 토큰 사용량을 크게 줄이면서도 대화의 핵심 문맥을 유지할 수 있으며, 특히 장시간 진행되는 대화에서 효과적입니다.

In [25]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import List
    
class SummarizedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    summary_threshold: int = Field(default=6)  # 요약을 시작할 메시지 수
    # 요약에 사용할 LLM - lambda를 사용하여 필드가 초기화될 때마다 새로운 ChatOpenAI 인스턴스 생성
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, top_p=0.9))  
    
    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        self.messages.extend(new_messages)

        print(f"메시지 수: {len(self.messages)}")
        
        # 메시지 수가 임계값을 넘으면 요약 수행
        if len(self.messages) >= self.summary_threshold:
            # 마지막 사용자 메시지 저장 (HumanMessage, AIMessage 순서로 생성)
            last_user_message = self.messages[-2]
            last_ai_message = self.messages[-1]
            
            # 요약 생성
            summary_prompt = (
                "Distill the above chat messages into a single summary message. "
                "Include as many specific details as you can."
                "Use the original language and tone of the conversation."
            )
            
            summary_chain_messages = [
                SystemMessage(content=(
                    "You are a helpful assistant. "
                    "Your task is to summarize the conversation accurately."
                )),
                *self.messages[:-2],  # 마지막 대화 턴을 제외한 모든 메시지
                HumanMessage(content=summary_prompt)
            ]
            
            # 요약 생성
            summary = self.llm.invoke(summary_chain_messages)
            
            # 메시지 리스트 초기화 후 요약과 마지막 메시지 추가
            self.messages = [
                summary,
                last_user_message,
                last_ai_message
            ]
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 요약된 히스토리 가져오기
def get_summarized_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = SummarizedInMemoryHistory()
    return store[session_id]


# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 체인 구성
chain_with_summarized_history = RunnableWithMessageHistory(
    chain,
    get_summarized_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(user_1)를 사용하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 2
여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.
2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 골목길 산책하며 한국의 옛 분위기를 느껴보세요.
3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋아요.
4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있고 야경이 특히 아름답습니다.
5. 홍대 – 젊음의 거리로, 예술과 음악, 카페 문화가 활발한 곳입니다.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.
7. 한강공원 – 강변을 따라 산책하거나 자전거 타기, 피크닉을 즐기기에 좋은 장소입니다.

필요하시면 교통편이나 맛집 정보도 알려드릴게요!


In [26]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.\n2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 골목길 산책하며 한국의 옛 분위기를 느껴보세요.\n3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋아요.\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있고 야경이 특히 아름답습니다.\n5. 홍대 – 젊음의 거리로, 예술과 음악, 카페 문화가 활발한 곳입니다.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.\n7. 한강공원 – 강변을 따라 산책하거나 자전거 타기, 피크닉을 즐기기에 좋은 장소입니다.\n\n필요하시면 교통편이나 맛집 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 40, 'total_tokens': 316, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id

In [27]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 4
여행 가이드 답변:
이전에 추천드린 장소 중에서 가장 인기 있는 곳은 경복궁과 N서울타워입니다.

- **경복궁**은 한국의 역사와 문화를 체험할 수 있는 대표적인 관광지로, 특히 외국인 관광객들에게 매우 인기가 많습니다. 한복을 입고 방문하면 무료 입장이 가능해 더욱 많은 분들이 찾고 있어요.

- **N서울타워**는 서울의 랜드마크 중 하나로, 서울 전경과 야경을 감상하기에 최적의 장소입니다. 특히 연인들의 데이트 코스로도 유명해 방문객이 많습니다.

두 곳 모두 서울 방문 시 꼭 들러볼 만한 인기 명소입니다! 필요하시면 입장 시간, 요금, 교통편도 안내해드릴게요.


In [28]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.\n2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 골목길 산책하며 한국의 옛 분위기를 느껴보세요.\n3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋아요.\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있고 야경이 특히 아름답습니다.\n5. 홍대 – 젊음의 거리로, 예술과 음악, 카페 문화가 활발한 곳입니다.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.\n7. 한강공원 – 강변을 따라 산책하거나 자전거 타기, 피크닉을 즐기기에 좋은 장소입니다.\n\n필요하시면 교통편이나 맛집 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 40, 'total_tokens': 316, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id

In [29]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "전주에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 6
여행 가이드 답변:
전주에서 가볼 만한 곳을 추천해드릴게요!

1. 전주한옥마을 – 전통 한옥이 잘 보존된 마을로, 한국 전통문화와 음식을 체험하기에 최적의 장소입니다. 한복 체험, 전통 공예 체험도 즐길 수 있어요.
2. 경기전 – 조선 태조 이성계의 초상화를 모신 역사적인 장소로, 전주한옥마을 내에 위치해 있습니다.
3. 전동성당 – 고딕 양식의 아름다운 성당으로, 전주의 대표적인 근대 건축물 중 하나입니다.
4. 오목대와 이목대 – 전주를 한눈에 내려다볼 수 있는 전망 명소로, 산책하기 좋습니다.
5. 남부시장 – 전주의 맛집과 특산품을 즐길 수 있는 전통 시장입니다. 특히 전주비빔밥과 다양한 길거리 음식이 유명해요.
6. 전주향교 – 조선시대 교육기관으로, 고즈넉한 분위기 속에서 한국 전통 문화를 느낄 수 있습니다.

필요하시면 교통편이나 맛집 추천도 도와드릴게요!


In [30]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 인기 장소로는 경복궁과 N서울타워를 추천드립니다. 경복궁은 조선 시대 대표 궁궐로 전통 건축과 정원을 감상할 수 있고, 한복 체험 시 무료 입장이 가능해 외국인 관광객에게 특히 인기가 많아요. N서울타워는 남산 정상에 위치한 전망대로 서울 전경과 야경이 아름다워 연인들의 데이트 코스로도 유명합니다. 이 외에도 북촌 한옥마을, 명동, 홍대, 동대문 디자인 플라자(DDP), 한강공원 등 다양한 매력적인 장소들이 있으니 필요하시면 교통편이나 맛집 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 546, 'total_tokens': 707, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEavzahRdsuJSFVOO23OtOgMGbyTG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--282de233-7433-4b9d-8402-7a8cfff61275-0', usage_metadata={'input_tokens': 546, 'output_tokens': 161, 'total_tokens': 707, 'input_token_details': {'audio

In [31]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 서로 비슷한 장소가 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 5
여행 가이드 답변:
서울과 전주에서 비슷한 성격을 가진 장소들을 비교해 드릴게요:

1. **전통 한옥과 궁궐**
   - 서울: **경복궁** — 조선 왕조의 대표 궁궐로, 전통 건축과 왕실 문화를 체험할 수 있습니다.
   - 전주: **전주한옥마을** — 전통 한옥이 모여 있는 마을로, 조선 시대 일반 서민과 양반의 생활상을 느낄 수 있어요.

2. **역사적 건축물**
   - 서울: **경기전** (전주에 위치)와 비슷한 역사적 장소로는 서울의 **창덕궁**이나 **덕수궁**이 있습니다.
   - 전주: **경기전** — 조선 태조 이성계의 초상화를 모신 역사적 건축물입니다.

3. **전통 시장과 음식**
   - 서울: **남대문시장**, **광장시장** 등 전통 시장에서 다양한 길거리 음식과 특산품을 즐길 수 있습니다.
   - 전주: **남부시장** — 전주비빔밥 등 지역 특산 음식을 맛볼 수 있는 전통 시장입니다.

4. **전망 명소**
   - 서울: **N서울타워** — 도시 전경과 야경을 감상할 수 있는 전망대입니다.
   - 전주: **오목대와 이목대** — 전주 시내를 한눈에 볼 수 있는 전망 명소입니다.

요약하자면, 서울의 경복궁과 전주의 전주한옥마을은 전통 건축과 문화를 체험할 수 있는 대표적인 장소이며, 두 도시 모두 전통 시장과 전망 명소가 있어 비슷한 관광 경험을 제공합니다. 필요하시면 더 자세한 비교나 여행 계획도 도와드릴게요!


In [32]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 인기 장소로는 경복궁과 N서울타워를 추천드립니다. 경복궁은 조선 시대 대표 궁궐로 전통 건축과 정원을 감상할 수 있고, 한복 체험 시 무료 입장이 가능해 외국인 관광객에게 특히 인기가 많아요. N서울타워는 남산 정상에 위치한 전망대로 서울 전경과 야경이 아름다워 연인들의 데이트 코스로도 유명합니다. 이 외에도 북촌 한옥마을, 명동, 홍대, 동대문 디자인 플라자(DDP), 한강공원 등 다양한 매력적인 장소들이 있으니 필요하시면 교통편이나 맛집 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 546, 'total_tokens': 707, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEavzahRdsuJSFVOO23OtOgMGbyTG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--282de233-7433-4b9d-8402-7a8cfff61275-0', usage_metadata={'input_tokens': 546, 'output_tokens': 161, 'total_tokens': 707, 'input_token_details': {'audio

In [33]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 각각 한 장소만 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 7
여행 가이드 답변:
서울에서는 **경복궁**을 추천드립니다. 한국 전통 궁궐의 아름다움과 역사적 의미를 깊이 느낄 수 있고, 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.

전주에서는 **전주한옥마을**을 추천해요. 전통 한옥 사이를 걸으며 한국의 전통 문화와 맛있는 전주비빔밥 등 지역 음식을 즐길 수 있는 최고의 장소입니다.

두 곳 모두 한국의 전통과 문화를 체험하기에 아주 좋은 선택입니다! 여행 일정에 참고하세요.


In [34]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울과 전주에서 가볼 만한 인기 장소를 추천드릴게요. 서울에서는 조선 시대 대표 궁궐인 경복궁과 남산 정상의 N서울타워가 유명하며, 경복궁에서는 한복 체험 시 무료 입장이 가능해 전통 건축과 문화를 즐길 수 있어요. 전주에서는 전통 한옥이 잘 보존된 전주한옥마을이 대표적이며, 한복 체험과 전통 공예 체험도 가능합니다. 또한 경기전(조선 태조 이성계 초상화 봉안), 전동성당(고딕 양식 성당), 오목대와 이목대(전망 명소), 남부시장(전주비빔밥 등 맛집과 특산품), 전주향교(조선시대 교육기관)도 추천드립니다. 서울과 전주를 비교하면, 경복궁과 전주한옥마을이 전통 건축과 문화를 체험할 수 있는 비슷한 장소이고, 남대문시장·광장시장과 전주 남부시장 모두 전통 시장과 길거리 음식으로 유명합니다. 전망 명소로는 서울의 N서울타워와 전주의 오목대·이목대가 각각 도시 전경을 감상할 수 있는 곳입니다. 필요하시면 교통편이나 맛집 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_tokens': 945, 'total_tokens': 1247, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_ed35aa4f5f', 'id': 'chatcmpl-CEawAARCpXjmAEsukG3itAjRu7kvi', 'service_tier': 'default', 'f

In [35]:
# 대화 히스토리 초기화
history.clear()

In [36]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[]


---
# **[실습]**

- 메시지 트리밍과 대화 요약 저장을 결합하여 메시지를 관리하는 기능을 구현합니다. 

1. 새로운 클래스 구조

In [41]:
class TrimmedAndSummarizedHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage]
    max_tokens: int  # 트리밍 기준
    summary_threshold: int  # 요약 기준
    llm: ChatOpenAI
    summarized_messages: List[BaseMessage]  # 요약된 메시지 저장용

2. 메시지 처리 로직
- 새 메시지 추가시 max_tokens 체크
- 트리밍 발생하면 제거될 메시지 식별
- 제거 예정 메시지들은 요약하여 summarized_messages에 저장
- 현재 메시지는 트리밍된 상태로 유지

3. 요약 프로세스
- 트리밍으로 제거될 메시지들만 선별
- 선별된 메시지들에 대해 summary_chain 실행
- 요약본을 시스템 메시지로 변환하여 저장

In [45]:
# ...existing code...

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, trim_messages, SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import List

class TrimmedAndSummarizedHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)  # 메시지 필드명을 그대로 유지 (속성명 변경 X)
    max_tokens: int = Field(default=2)  # 트리밍 기준
    summary_threshold: int = Field(default=6)  # 요약 기준 (실습에서는 트리밍 시 요약)
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, top_p=0.9))
    summarized_messages: List[BaseMessage] = Field(default_factory=list)  # 요약된 메시지 저장용
    
    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        # 새 메시지 추가
        self.messages.extend(new_messages)
        
        # 트리밍 수행
        trimmer = trim_messages(
            strategy="last",
            max_tokens=self.max_tokens,
            token_counter=len
        )
        trimmed_messages = trimmer.invoke(self.messages)
        
        # 제거될 메시지 식별 (트리밍으로 제거되는 메시지들)
        removed_messages = self.messages[:-len(trimmed_messages)] if len(self.messages) > len(trimmed_messages) else []
        
        # 제거될 메시지가 있으면 요약 수행
        if removed_messages:
            summary_prompt = (
                "Distill the above chat messages into a single summary message. "
                "Include as many specific details as you can. "
                "Use the original language and tone of the conversation."
            )
            
            summary_chain_messages = [
                SystemMessage(content=(
                    "You are a helpful assistant. "
                    "Your task is to summarize the conversation accurately."
                )),
                *removed_messages,
                HumanMessage(content=summary_prompt)
            ]
            
            # 요약 생성
            summary_response = self.llm.invoke(summary_chain_messages)
            summary_message = SystemMessage(content=summary_response.content)
            
            # 요약을 summarized_messages에 추가
            self.summarized_messages.append(summary_message)
        
        # 현재 메시지를 트리밍된 상태로 유지
        self.messages = trimmed_messages
    
    def clear(self) -> None:
        self.messages = []
        self.summarized_messages = []
    
    def get_all_messages(self) -> List[BaseMessage]:
        # 요약된 메시지와 현재 메시지를 결합하여 반환
        return self.summarized_messages + self.messages

# 세션 저장소
store = {}

# 세션 ID로 트리밍 및 요약된 히스토리 가져오기
def get_trimmed_summarized_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = TrimmedAndSummarizedHistory()
    return store[session_id]

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 트리밍 및 요약된 히스토리 관리 추가
chain_with_trimmed_summarized_history = RunnableWithMessageHistory(
    chain,
    get_trimmed_summarized_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(user_1)를 사용하여 체인 실행
response = chain_with_trimmed_summarized_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

# 대화 히스토리 출력
history = get_trimmed_summarized_session_history("user_1")
all_messages = history.get_all_messages()
print("\n대화 히스토리:")
pprint(all_messages)

# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_summarized_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"\n여행 가이드 답변:\n{response.content}")

# 대화 히스토리 출력
history = get_trimmed_summarized_session_history("user_1")
all_messages = history.get_all_messages()
print("\n대화 히스토리:")
pprint(all_messages)

# 추가 질문으로 체인 실행 (트리밍 및 요약 테스트)
response = chain_with_trimmed_summarized_history.invoke(
    {
        "input": "이 장소의 정식 명칭은 무엇인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"\n여행 가이드 답변:\n{response.content}")

# 대화 히스토리 출력
history = get_trimmed_summarized_session_history("user_1")
all_messages = history.get_all_messages()
print("\n대화 히스토리:")
pprint(all_messages)

# 히스토리 초기화
history.clear()

# 초기화 후 히스토리 출력
print("\n초기화 후 히스토리:")

여행 가이드 답변:
서울에는 볼거리와 즐길 거리가 정말 많아요! 몇 가지 추천해드릴게요.

1. 경복궁 – 조선 시대의 대표 궁궐로, 한복 체험도 할 수 있어요.
2. 북촌 한옥마을 – 전통 한옥이 모여 있는 곳으로, 사진 찍기 좋아요.
3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가입니다.
4. N서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대예요.
5. 홍대 – 젊음의 거리로 카페, 공연, 예술 공간이 많아요.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 야간 조명이 멋져요.
7. 한강공원 – 자전거 타기, 피크닉, 야경 감상하기 좋은 장소입니다.

관심사나 일정에 따라 더 구체적으로 추천해드릴 수 있으니, 궁금한 점 있으면 말씀해 주세요!

대화 히스토리:
[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에는 볼거리와 즐길 거리가 정말 많아요! 몇 가지 추천해드릴게요.\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 한복 체험도 할 수 있어요.\n2. 북촌 한옥마을 – 전통 한옥이 모여 있는 곳으로, 사진 찍기 좋아요.\n3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가입니다.\n4. N서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대예요.\n5. 홍대 – 젊음의 거리로 카페, 공연, 예술 공간이 많아요.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 야간 조명이 멋져요.\n7. 한강공원 – 자전거 타기, 피크닉, 야경 감상하기 좋은 장소입니다.\n\n관심사나 일정에 따라 더 구체적으로 추천해드릴 수 있으니, 궁금한 점 있으면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 224, 'prompt_tokens': 40, 'total_to